## OpenAI's chat completions API


OpenAI's chat models take a list of messages as input and return a model-generated message as output.
The chat format is designed to make multi-turn conversations easy.
But it is just as useful for single-turn tasks without any conversation.

### The message object

The message message object records the contributions to a conversation in a [**list**](link):

```json
[
    <first message>,
    <second message>,
    ...,
    <last message>
]
```

Each list element is a [**dictionary**](link) with elements **_role_** and **_content_**:

```json
{
    "role": "system" | "user" | "assistant", // <== choose one
    "content": <string> // <== insert text
}
```

##### *Example*

Below, we mimic the tweet sentiment classifier in OpenAI's playground:

In [6]:
instructions = """
Act like a sentiment classification system.

You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.

Categorize the text enclosed in triple quotes into one of the following categories: "positive", "neutral", "negative"
"""

messages = [
    {
        "role": "system",
        "content": instructions.strip()
    },
    {
        "role": "user",
        "content": "'''I loved the new Batman movie!'''"
    },
]

In [7]:
messages

[{'role': 'system',
  'content': 'Act like a sentiment classification system.\n\nYou will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.\n\nCategorize the text enclosed in triple quotes into one of the following categories: "positive", "neutral", "negative"'},
 {'role': 'user', 'content': "'''I loved the new Batman movie!'''"}]

#### Roles

There are three **roles**:

- **"system"**: a 'hidden' prompt to provide the model with relevant task-specific information.
- **"user"**: the user who is sending inputs and requests to the model
- **"assistant"**: the model who is responding to the user

**_Note:_** When you use ChatGPT in the browser, you only see the messages send by you (the user) and the model's responses (the assistants message).

Here is a typical chat conversation from a translation example:

```json
[
    {
        "role": "system",
        "content": "Act as a translation system that translates English texts to French"
    },
    {
        "role": "user",
        "content": "Hello, how are you?"
    },
    {
        "role": "assistant",
        "content": "Bonjour, comment vas-tu?"
    }
]
```

#### The system message

The system message is only defined once at the beginning of a conversation.
It provides the LLM ("assistant") with behavioral instructions.
That is, it sets the behavior of the assistant.
For example, you can modify the personality ("persona") of the assistant or provide specific instructions about how it should behave throughout the conversation.

Note, however that the system message is optional.
The model's behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."

#### User and assistant messages

After the system message, conversations can be as short as one message or include many back and forth turns between the user and the assistant.
The user messages provide requests or comments for the assistant to respond to.
If after the system message you only include one user message, the model will return an assistant message &mdash; the model's response to the user's message (given the instructions in the system message).

But as assistant messages store previous assistant responses, you can also write them to give examples of desired behavior.
Including conversation history is important when user instructions refer to prior messages.
Because chat models have no memory of past requests, all relevant information must be supplied as part of the conversation history in each request.

### Requesting a chat completion

The way you use the chat completions API for text coding is by specifying a system prompt and a user input and requesting a model-generated assisant response.

To allow communication with the OpenAI API, you first need to create a client that takes care of sending API requests and receiving responses:

In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

Now we can define a message object and request a model-generated response:

In [9]:
messages = [
    {
        "role": "system",
        "content": "Act as a translation system that translates English texts to French"
    },
    {
        "role": "user",
        "content": "Hello, how are you?"
    }
]

Together with the identifier of the model we want to ask for a response, we send the message object to the API and receive a response from the model: 

In [10]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

The `response` object returned by calling `chat.completion.create` is a little complex.
The relevant information is stored in the `choices` list, which contains a single element by default:

In [13]:
len(response.choices)

1

In [14]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Bonjour, comment ça va ?', refusal=None, role='assistant', function_call=None, tool_calls=None))

As you can see, this is a `Choices` object that contains, among others, a `message` atttribute that contains the content of the assistant's response:

In [16]:
response.choices[0].message.content

'Bonjour, comment ça va ?'

#### Text classification example

In our applications, we apply this logic to task a GPT model to classify texts according to our coding instructions and coding scheme.

In [17]:
messages = [ 
    # system prompt: coding instruction and specify coding scheme
    {
        "role": "system", 
        "content": (
            "Act as a sentiment classification system that classifies the sentiment of a given text. "
            "Classify the text in the input into one of the following categories: positive, negative, neutral. "
            "Only respond with the chosen category and no further text."
        )
    },
    # user prompt: a single text to be classified
    {"role": "user", "content": "I am so happy today!"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

response.choices[0].message.content

'positive'

In [32]:
instructions = """
You will be provided with a tweet, and your task is to classify its sentiment as positive, neutral, or negative.

Categorize the text enclosed in triple quotes into one of the following categories: positive, neutral, negative
"""

text = "I loved the new Batman movie!"
messages = [ 
    {
        "role": "system", 
        "content": instructions # <== copy-paste your custom instructions here
    },
    {"role": "user", "content": f"'''{text}'''"} # <== copy-paste your text here
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    logprobs=True,
    top_logprobs=20,
)

response.choices[0].message.content

'Positive'

In [33]:
[(lp.token, lp.logprob) for lp in response.choices[0].logprobs.content[0].top_logprobs]

[('Positive', -0.00420804),
 ('positive', -5.504208),
 ('The', -9.254208),
 ('**', -10.379208),
 ('Sent', -13.629208),
 ('This', -14.629208),
 ('_positive', -14.879208),
 ('Category', -15.004208),
 ('```', -15.629208),
 (' positive', -15.754208),
 ('``', -16.754208),
 (' Positive', -16.754208),
 ('-', -17.254208),
 ('Pos', -17.379208),
 ('Categor', -17.754208),
 ('\n', -18.129208),
 ('Classification', -18.129208),
 ('###', -18.254208),
 ('"', -18.379208),
 ('***', -18.504208)]